Initialize Gen and the function libraries

In [1]:
using Gen

In [25]:
cd("/Users/lilis/Documents/GitHub/HSOManalysisPackages/JuliaLanguage_Hierarchical_Models")
include(join([pwd(), "/function_library/file_modules.jl"]))
refresh_tools("function_library")
refresh_tools("gen_stepvramp_library")

Refreshed (2021-03-29_2026) GLMNet_tools.jl auto_pull_matlabdatasets.jl cluster_modules.jl feedback_modules.jl file_modules.jl hazard_modules.jl v 11:37
math_modules.jl modeling_tools.jl plotting_tools.jl probabilistic_modules.jl Refreshed (2021-03-29_2026) gen_math_modules.jl gen_plotting_modules.jl gen_stepvramp.jl runner_modules.jl svr_filemods.jl 

Enter the CSV folder for all our sessions

In [9]:
# cd("N:\\Assad Lab\\allison\\Collated Analyses\\JULIA_CSVs\\All_SNc_nosplitsesh")
cd("/Volumes/Neurobio/Assad Lab/allison/Collated Analyses/JULIA_CSVs/All_SNc_nosplitsesh")

In [22]:
B1_idx = 1:5
B2_idx = 6:13
B3_idx = 14:24
B5_idx = 75:80
B6_idx = 25:30
H3_idx = 44:51
H4_idx = 52:66
H5_idx = 67:74
H6_idx = 81:92
H7_idx = 93:98
H14_idx = 31:38
H15_idx = 39:43

39:43

In [ ]:
runID = "2021_3_29_Mac"
pathIDx = B5_idx
# mac
collatedPath = "/Volumes/Neurobio/Assad Lab/allison/Collated Analyses/JULIA_CSVs/All_SNc_nosplitsesh"
compositesavepath="/Volumes/Neurobio/Assad Lab/allison/Collated Analyses/HIER results/3-26-21 hierarchy1"
# windows
# collatedPath = "N:\\Assad Lab\\allison\\Collated Analyses\\JULIA_CSVs\\All_SNc_nosplitsesh"
# compositesavepath="N:\\Assad Lab\\allison\\Collated Analyses\\HIERARCHY RESULTS\\3-26-21 hierarchy1"
modelpackagefunction = hierarchy_v1#SHUFFLE_DAHxmodel_logit_200hx_pkg
postprocessingfunction = template_postprocessingfunction#bootlogit_timeslice_postprocessingfunction1

(fails, results,postprocessing) = run_collated_model(collatedPath, 
    modelpackagefunction; pathIDx = pathIDx,runFails=false, failDirs=[], postprocessingfunction=postprocessingfunction,
    compositesavepath=compositesavepath, runID=runID, suppressFigures=true); 

-----------------------------------------------
	
Initializing run_collated_model for hierarchy_v1_2021_3_29_Mac
	
Found 98 sessions in /Volumes/Neurobio/Assad Lab/allison/Collated Analyses/JULIA_CSVs/All_SNc_nosplitsesh:
1. B1_SNc_13
2. B1_SNc_15
3. B1_SNc_18
4. B1_SNc_19
5. B1_SNc_20
6. B2_SNc_11
7. B2_SNc_15
8. B2_SNc_17
9. B2_SNc_18
10. B2_SNc_19
11. B2_SNc_20
12. B2_SNc_21
13. B2_SNc_9
14. B3_SNc_10
15. B3_SNc_12
16. B3_SNc_14
17. B3_SNc_15
18. B3_SNc_16
19. B3_SNc_17
20. B3_SNc_18
21. B3_SNc_19
22. B3_SNc_20
23. B3_SNc_21
24. B3_SNc_8
25. B6_SNc_10
26. B6_SNc_11
27. B6_SNc_12
28. B6_SNc_13
29. B6_SNc_7
30. B6_SNc_9
31. H14_SNc_10
32. H14_SNc_12
33. H14_SNc_14
34. H14_SNc_16
35. H14_SNc_18
36. H14_SNc_5
37. H14_SNc_7
38. H14_SNc_8
39. H15_SNc_16
40. H15_SNc_18
41. H15_SNc_5
42. H15_SNc_7
43. H15_SNc_8
44. H3_SNc_13
45. H3_SNc_15
46. H3_SNc_17
47. H3_SNc_18
48. H3_SNc_19
49. H3_SNc_20
50. H3_SNc_7
51. H3_SNc_9
52. H4_SNc_11
53. H4_SNc_13
54. H4_SNc_15
55. H4_SNc_17
56. H4_SNc_18
57

! - WARNING:failed to complete inference function on trial 155
! - SoftERROR:reducing over an empty collection is not allowed!
! - WARNING:failed to complete inference function on trial 164
! - SoftERROR:reducing over an empty collection is not allowed!
! - WARNING:failed to complete inference function on trial 168
! - SoftERROR:reducing over an empty collection is not allowed!
! - WARNING:failed to complete inference function on trial 1
! - SoftERROR:reducing over an empty collection is not allowed!
! - WARNING:failed to complete inference function on trial 206
! - SoftERROR:reducing over an empty collection is not allowed!
! - WARNING:failed to complete inference function on trial 239
! - SoftERROR:reducing over an empty collection is not allowed!
! - WARNING:failed to complete inference function on trial 262
! - SoftERROR:reducing over an empty collection is not allowed!
   *==-------- (141/705) 03/29/21 22:06:654
! - WARNING:failed to complete inference function on trial 284
! - So

! - WARNING:failed to complete inference function on trial 1279
! - SoftERROR:$(Expr(:escape, :(ccall(#= /Users/lilis/.julia/packages/PyCall/tqyST/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))!
! - WARNING:failed to complete inference function on trial 127
! - SoftERROR:$(Expr(:escape, :(ccall(#= /Users/lilis/.julia/packages/PyCall/tqyST/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))!
! - WARNING:failed to complete inference function on trial 1280
! - SoftERROR:$(Expr(:escape, :(ccall(#= /Users/lilis/.julia/packages/PyCall/tqyST/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))!
! - WARNING:failed to complete inference function on trial 1281
! - SoftERROR:$(Expr(:escape, :(ccall(#= /Users/lilis/.julia/packages/PyCall/tqyST/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))!
! - WARNING:failed to

! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 160
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 161
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 162
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 163
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 164
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 165
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 167
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 168
! - SoftERROR:cwd: no such file or directory (ENOENT)!
!

! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 250
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 251
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 253
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 254
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 255
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 257
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 258
! - SoftERROR:cwd: no such file or directory (ENOENT)!
! - WARNING:failed to complete inference function on trial 259
! - SoftERROR:cwd: no such file or directory (ENOENT)!
!